In [1]:
import pandas

# Prob1 is the same problem as Module 2A
url = 'https://raw.githubusercontent.com/acedesci/scanalytics/master/EN/S08_09_Retail_Analytics/predictedSales_Prob1.csv'

# Prob2 is the large-scale problem
# url = 'https://raw.githubusercontent.com/acedesci/scanalytics/master/EN/S08_09_Retail_Analytics/predictedSales_Prob2.csv'

predDemand = pandas.read_csv(url)

# Dataset is now stored in a Pandas Dataframe predDemand
predDemand

,NA,avgPriceChoice,UPC,PRICE,PRICE_p2,FEATURE,DISPLAY,TPR_ONLY,RELPRICE,predictSales
0,0,3.0,1600027528,2.5,6.25,0,0,0,0.833333,95.0
1,1,3.0,1600027528,3.0,9.00,0,0,0,1.000000,67.0
2,2,3.0,1600027528,3.5,12.25,0,0,0,1.166667,46.0
3,3,3.0,1600027564,2.5,6.25,0,0,0,0.833333,24.0
4,4,3.0,1600027564,3.0,9.00,0,0,0,1.000000,23.0
5,5,3.0,1600027564,3.5,12.25,0,0,0,1.166667,20.0
6,6,3.0,3000006340,2.5,6.25,0,0,0,0.833333,6.0
7,7,3.0,3000006340,3.0,9.00,0,0,0,1.000000,4.0
8,8,3.0,3000006340,3.5,12.25,0,0,0,1.166667,3.0
9,9,3.0,3800031829,2.5,6.25,0,0,0,0.833333,33.0


In [2]:
avgPriceList = predDemand['avgPriceChoice'].unique()
inputColumns = ['avgPriceChoice', 'UPC', 'PRICE','predictSales']
print("Possible average price choices (k/N.Product):"+str(avgPriceList))


Possible average price choices (k/N.Product):[3.]


In [3]:
# Nere we choose which value of k (avgPriceValue x N. of products) we would like to use in the model
# Note that k must be among the choices where the prediction has been prepared
avgPriceValue =  avgPriceList[0]

# Now we select only the row which corresponds to the previously chosen value of avgPriceValue (again k = avgPriceValue x N. of products)
predDemand_k = predDemand.loc[predDemand['avgPriceChoice'] == avgPriceValue][inputColumns]
print(predDemand_k)
productList = predDemand_k['UPC'].unique()
priceList = predDemand_k['PRICE'].unique()

# Here we prepare the dictionary to be used in the optimization model
p = {}
D = {}

for upc in productList:
  for price in priceList:
    p[(upc,price)] = price
    D[(upc,price)] = predDemand_k.loc[(predDemand['UPC'] == upc) & (predDemand_k['PRICE'] == price)]['predictSales'].values[0]

print(p)
print(D)

    avgPriceChoice         UPC  PRICE  predictSales
0              3.0  1600027528    2.5          95.0
1              3.0  1600027528    3.0          67.0
2              3.0  1600027528    3.5          46.0
3              3.0  1600027564    2.5          24.0
4              3.0  1600027564    3.0          23.0
5              3.0  1600027564    3.5          20.0
6              3.0  3000006340    2.5           6.0
7              3.0  3000006340    3.0           4.0
8              3.0  3000006340    3.5           3.0
9              3.0  3800031829    2.5          33.0
10             3.0  3800031829    3.0          24.0
11             3.0  3800031829    3.5          20.0
{(1600027528, 2.5): 2.5, (1600027528, 3.0): 3.0, (1600027528, 3.5): 3.5, (1600027564, 2.5): 2.5, (1600027564, 3.0): 3.0, (1600027564, 3.5): 3.5, (3000006340, 2.5): 2.5, (3000006340, 3.0): 3.0, (3000006340, 3.5): 3.5, (3800031829, 2.5): 2.5, (3800031829, 3.0): 3.0, (3800031829, 3.5): 3.5}
{(1600027528, 2.5): 95.0, (16000275

In [4]:
from pyomo.environ import *

iIndexList = list(range(len(productList)))
jIndexList = list(range(len(priceList)))

model = ConcreteModel()
# Variables
model.x = Var(productList, priceList, within = Binary)

# Constraints
model.PriceChoiceUPC = ConstraintList()
model.sumPrice = ConstraintList()

# Print to review the model (equations are still not included)
model.pprint()

5 Set Declarations
    PriceChoiceUPC_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    0 :      {}
    sumPrice_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    0 :      {}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   12 : {(1600027528, 2.5), (1600027528, 3.0), (1600027528, 3.5), (1600027564, 2.5), (1600027564, 3.0), (1600027564, 3.5), (3000006340, 2.5), (3000006340, 3.0), (3000006340, 3.5), (3800031829, 2.5), (3800031829, 3.0), (3800031829, 3.5)}
    x_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1600027528, 1600027564, 3000006340, 3800031829}
    x_index_1 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Member

In [5]:
# Objective function

obj_expr = sum(p[(i,j)]*D[(i,j)]*model.x[i,j] for i in productList for j in priceList)
print(obj_expr)
model.OBJ = Objective(expr = obj_expr, sense = maximize)

237.5*x[1600027528,2.5] + 201.0*x[1600027528,3.0] + 161.0*x[1600027528,3.5] + 60.0*x[1600027564,2.5] + 69.0*x[1600027564,3.0] + 70.0*x[1600027564,3.5] + 15.0*x[3000006340,2.5] + 12.0*x[3000006340,3.0] + 10.5*x[3000006340,3.5] + 82.5*x[3800031829,2.5] + 72.0*x[3800031829,3.0] + 70.0*x[3800031829,3.5]


In [6]:
# Constraints #1
for i in productList:
  const1_expr = sum(model.x[i,j] for j in priceList) == 1
  print(const1_expr)
  model.PriceChoiceUPC.add(expr = const1_expr)


x[1600027528,2.5] + x[1600027528,3.0] + x[1600027528,3.5]  ==  1
x[1600027564,2.5] + x[1600027564,3.0] + x[1600027564,3.5]  ==  1
x[3000006340,2.5] + x[3000006340,3.0] + x[3000006340,3.5]  ==  1
x[3800031829,2.5] + x[3800031829,3.0] + x[3800031829,3.5]  ==  1


In [7]:
# Constraints #2
const2_expr = sum(p[i,j]*model.x[i,j] for i in productList for j in priceList) == avgPriceValue*len(productList)
print(const2_expr)
model.sumPrice.add(expr = const2_expr)



2.5*x[1600027528,2.5] + 3.0*x[1600027528,3.0] + 3.5*x[1600027528,3.5] + 2.5*x[1600027564,2.5] + 3.0*x[1600027564,3.0] + 3.5*x[1600027564,3.5] + 2.5*x[3000006340,2.5] + 3.0*x[3000006340,3.0] + 3.5*x[3000006340,3.5] + 2.5*x[3800031829,2.5] + 3.0*x[3800031829,3.0] + 3.5*x[3800031829,3.5]  ==  12.0


In [8]:
model.pprint()

5 Set Declarations
    PriceChoiceUPC_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    sumPrice_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :    {1,}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   12 : {(1600027528, 2.5), (1600027528, 3.0), (1600027528, 3.5), (1600027564, 2.5), (1600027564, 3.0), (1600027564, 3.5), (3000006340, 2.5), (3000006340, 3.0), (3000006340, 3.5), (3800031829, 2.5), (3800031829, 3.0), (3800031829, 3.5)}
    x_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1600027528, 1600027564, 3000006340, 3800031829}
    x_index_1 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : M

In [9]:
# Solve the model
opt = SolverFactory('glpk')
opt.solve(model)

model.display()

Model unknown

  Variables:
    x : Size=12, Index=x_index
        Key               : Lower : Value : Upper : Fixed : Stale : Domain
        (1600027528, 2.5) :     0 :   1.0 :     1 : False : False : Binary
        (1600027528, 3.0) :     0 :   0.0 :     1 : False : False : Binary
        (1600027528, 3.5) :     0 :   0.0 :     1 : False : False : Binary
        (1600027564, 2.5) :     0 :   0.0 :     1 : False : False : Binary
        (1600027564, 3.0) :     0 :   0.0 :     1 : False : False : Binary
        (1600027564, 3.5) :     0 :   1.0 :     1 : False : False : Binary
        (3000006340, 2.5) :     0 :   0.0 :     1 : False : False : Binary
        (3000006340, 3.0) :     0 :   0.0 :     1 : False : False : Binary
        (3000006340, 3.5) :     0 :   1.0 :     1 : False : False : Binary
        (3800031829, 2.5) :     0 :   1.0 :     1 : False : False : Binary
        (3800031829, 3.0) :     0 :   0.0 :     1 : False : False : Binary
        (3800031829, 3.5) :     0 :   0.0